# Reducing features by stats modeling
DC 

## Procedure

* Regress each cpg with age
* get p-values for each cpg for slope  not equal to 0
* Adjust pvalues for multiple testing, FDR = 0.01 (BH)
* Select 100 cpgs with lowest adjusted p values
* Make models with these 
* Compare to the XGboost crossvalidation method


## Conclusions:
* Works about as wll as XGboost method slightly worse.
* After adjusting for multiple testing 177,993 cpgs significant of 406,628 were found significant (44%). 
* the two methods share 48 top100 cpgs in common

#### Imports

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('poster')
import os

#### Import our modules

In [2]:
from age_predict.Loading_EWAS_Aging_Data import *
from age_predict.Regression import *
from age_predict.pickle_unpickle import *

In [3]:
# Set working directory
os.chdir('/Users/dcox/Desktop/EWAS-organized')

#### set tissue of interest

In [4]:
tissue_of_interest = 'whole blood'

#### Load data

In [5]:
%%time
file_in = './data/age_methylation_v1/age_methylation_v1.txt'
file_out = './data/age_methylation_v1/'+ tissue_of_interest + '_age_methylation_v1.txt'

df_t = load_EWAS_aging_by_tissue(file_in, file_out, search_term=tissue_of_interest,
                          num_rows='all', verbose=True)

finished line 10000
finished line 20000
finished line 30000
finished line 40000
finished line 50000
finished line 60000
finished line 70000
finished line 80000
finished line 90000
finished line 100000
finished line 110000
finished line 120000
finished line 130000
finished line 140000
finished line 150000
finished line 160000
finished line 170000
finished line 180000
finished line 190000
finished line 200000
finished line 210000
finished line 220000
finished line 230000
finished line 240000
finished line 250000
finished line 260000
finished line 270000
finished line 280000
finished line 290000
finished line 300000
finished line 310000
finished line 320000
finished line 330000
finished line 340000
finished line 350000
finished line 360000
finished line 370000
finished line 380000
finished line 390000
finished line 400000
finished line 410000
finished line 420000
finished line 430000
finished line 440000
finished line 450000
finished line 460000
finished line 470000
finished line 480000



#### Select age 20 and older

In [6]:
df_t_over20 = df_t[df_t.age >=20]

#### Split and impute data

In [9]:
%%time
df_p_working, df_p_saved = splitting_and_imputing(df_t_over20, input_percent=10, 
                                           fraction_test=0.25, seed=2021)

CPU times: user 3min 55s, sys: 1min 5s, total: 5min 1s
Wall time: 6min 5s


## HERE STARTS THE STATS ANALYSIS

In [14]:
df_processed_imp_over20 = df_p_working

### Regressing each cpg on age

In [ ]:
%%time
import statsmodels.api as sm

p_values = []
slopes = []
cpgs = []
y = df_processed_imp_over20['age']
for i, cpg in enumerate(df_processed_imp_over20.columns[2:]):
    X = df_processed_imp_over20[cpg]
    X = sm.add_constant(X)
    fit = sm.OLS(y, X).fit()
    p_values.append(fit.pvalues[1])
    slopes.append(fit.params[1])
    if i % 10000 == 0:
        print(i, cpg, fit.pvalues[1], fit.params[1])

0 cg02494853 3.7694711013825014e-05 109.1102451144472
10000 cg18395233 2.5347216034277503e-05 -80.8727102275237
20000 cg22475904 0.5722415010614574 -56.08104271607084
30000 cg17378483 0.008523204061234543 -284.5575574787013
40000 cg09295542 0.06922861451114537 -25.45126344779563
50000 cg17495792 0.039582205592826596 -93.16145792532843
60000 cg03943218 9.126767073561638e-24 147.82120116102618
70000 cg18519694 0.00020673880674197142 40.47922122864131
80000 cg15988970 0.8185538849982108 1.3735697689378803
90000 cg03622875 0.6153501846111435 33.50495235777683
100000 cg19500275 0.6782779291948458 -21.14645466451135
110000 cg22598669 0.032182775862453356 -31.78830169271285
120000 cg09413198 0.008646430611345512 -23.159485353854855
130000 cg06840154 7.862391350796803e-29 -147.72077712489377
140000 cg17493193 0.25648836511167517 -31.642837546017603
150000 cg01479187 8.27942108389442e-16 -52.53628102782083
160000 cg16450590 0.03525714752596321 24.941430341570424


### Geting pvalues into a dataframe

In [ ]:
df_pvals = pd.DataFrame({'cpgs': df_processed_imp_over20.columns[2:], 'slopes': slopes, 'p_values': p_values})

In [ ]:
df_pvals

In [ ]:
# sorting 
df_pvals_sorted = df_pvals.sort_values('p_values', ascending=True)
df_pvals_sorted

### Getting adjusted p_values for Multiple testing FDR = 0.01 Benjamini Hochberg method

In [ ]:
from statsmodels.stats.multitest import fdrcorrection
rejected, q_values = fdrcorrection(df_pvals_sorted.p_values, alpha = 0.01, is_sorted=True)

In [ ]:
df_pvals_sorted['q_values_fdr1%'] = q_values
df_pvals_sorted['rejected'] = rejected

In [ ]:
df_pvals_sorted

In [ ]:
df_qvals_sorted = df_pvals_sorted.sort_values('q_values_fdr1%')

In [ ]:
df_qvals_sorted 

### How many cpgs significant?

In [ ]:
df_qvals_sorted.rejected.sum()

In [ ]:
cpgs_statsmod_blood_ranked = df_qvals_sorted.cpgs

In [ ]:
# Saving ramkings
import pickle
with open('./cpgs_ranked/cpgs_statsmod_blood_ranked', 'wb') as fp:
    pickle.dump(cpgs_statsmod_blood_ranked, fp)

In [ ]:
with open('./cpgs_ranked/cpgs_XGboost_whole_blood_ranked', 'rb') as fp:
    cpgs_XGboost_blood_ranked = pickle.load(fp)

In [ ]:
Top100_cpg_XGboost_blood = cpgs_XGboost_blood_ranked[:100]

### Count number of top 100 cpgs in common with the XGBoost cross val method

In [ ]:
count=0
cpgs_common = []
for i, cpg in enumerate(cpgs_statsmod_blood_ranked[:100]):
    if cpg in cpgs_XGboost_blood_ranked[:100]:
        print(i, cpg)
        cpgs_common.append(cpg)
        count+=1
print(count)

In [ ]:
Top100_cpgs_statsmodel_blood = cpgs_statsmod_blood_ranked[:100]
plt.rcdefaults()

### Modeling with the top 100 cpgs via stats modeling

In [ ]:
from sklearn.model_selection import train_test_split
X = df_p_working[Top100_cpgs_statsmodel_blood]
y = df_p_working.age
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 2021)

In [ ]:
plt.rcdefaults()
mod_100, rms_train, rms_test, r2_train, r2_test = linear_regress(X_train, y_train, X_test, y_test, plot=True)

In [ ]:
XG_100, rms_train, rms_test, r2_train, r2_test, feature_importances_ = xgboost_regress(X_train, y_train, X_test, y_test, early_stopping_rounds=10, plot=True)

In [ ]:
 mod_lasso_100, rms_train, rms_test, r2_train, r2_test = lassoCV_regress(X_train, y_train, X_test, y_test, plot=True, alphas=[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5],cv=5)

In [ ]:
mod_ridge_100, rms_train, rms_test, r2_train, r2_test = ridgeCV_regress(X_train, y_train, X_test, y_test, plot=True, alphas=[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5],cv=5)

In [ ]:
# Save models
import pickle

with open('./models/mod_100_blood_stats', 'wb') as fp:
    pickle.dump(mod_100, fp)
    
with open('./models/XG_100_blood_stats', 'wb') as fp:
    pickle.dump(XG_100, fp)
    
with open('./models/mod_lasso_100_blood_stats', 'wb') as fp:
    pickle.dump(mod_lasso_100, fp)
    
with open('./models/mod_ridge_100_blood_stats', 'wb') as fp:
    pickle.dump(mod_ridge_100, fp)

### Model Perfomance on Heldout Data

In [ ]:
X_saved = df_p_saved[Top100_cpgs_statsmodel_blood]
y_saved = df_p_saved.age

In [ ]:
plt.rcdefaults()

In [ ]:
preds, MSE, rms, r2, MAE, r_corr = test_model_on_heldout_data(X_saved, y_saved, mod_100, mtype='Linear Regression', figsize=(8,4), xlim=None)

In [ ]:
preds, MSE, rms, r2, MAE, r_corr = test_model_on_heldout_data(X_saved, y_saved, XG_100, mtype='XGboost Regression', figsize=(8,4), xlim=None)

In [ ]:
preds, MSE, rms, r2, MAE, r_corr = test_model_on_heldout_data(X_saved, y_saved, mod_ridge_100, mtype='Ridge Regression', figsize=(8,4), xlim=None)

In [ ]:
preds, MSE, rms, r2, MAE, r_corr = test_model_on_heldout_data(X_saved, y_saved, mod_lasso_100, mtype='Lasso Regression', figsize=(8,4), xlim=None)